### Par: HOUSNI Kamar

## Projet : Metaheuristic

### Constructive heuristic- Finite Best Strip

* We will start by reading our data;
    * we extract the bin and item dimensions in order to perform our heuristic algorithm on the data.

In [14]:
import matplotlib.pyplot as plt
import random
import time

In [2]:
def read_dataset(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Extract bin dimensions
    bin_width = None
    bin_height = None
    items = []
    for line in lines:
        if line.startswith('W:'):
            bin_width = int(line.split(':')[1].strip())
        elif line.startswith('H:'):
            bin_height = int(line.split(':')[1].strip())
        elif line.startswith('w:'):
            item_width = int(line.split(':')[1].strip())
            item_height = int(lines[lines.index(line) + 1].split(':')[1].strip())
            items.append((item_width, item_height))

    return bin_width, bin_height, items

In [19]:
def fbs_heuristic(bin_width, bin_height, items):
    items.sort(key=lambda x: x[1], reverse=True)
    strip_width = bin_width
    shelves = []
    for item in items:
        shelf_height = item[1]
        best_shelf = None
        best_residual_width = float('inf')
        for shelf in shelves:
            if shelf[0] >= item[0] and shelf[1] < best_residual_width:
                best_shelf = shelf
                best_residual_width = shelf[1]
        if best_shelf is None:
            shelves.append((item[0], strip_width - item[0]))
        else:
            shelves.remove(best_shelf)
            shelves.append((best_shelf[0], best_shelf[1] - item[0]))

    bins = []
    for shelf in shelves:
        best_bin = None
        best_residual_height = float('inf')
        for bin in bins:
            if bin[1] >= shelf[0] and bin[0] < best_residual_height:
                best_bin = bin
                best_residual_height = bin[0]
        if best_bin is None:
            bins.append((bin_height - shelf[0], strip_width))
        else:
            bins.remove(best_bin)
            bins.append((best_bin[0] - shelf[0], best_bin[1]))

    return len(bins), bins

In [20]:
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')
print(f"Bin dimensions: {bin_width} x {bin_height}")
print("Item dimensions:")
for item in items:
    print(f"{item[0]} x {item[1]}")

num_bins = fbs_heuristic(bin_width, bin_height, items)
print(f"Number of bins used: {num_bins}")

Bin dimensions: 10 x 10
Item dimensions:
5 x 9
4 x 2
10 x 6
5 x 9
6 x 3
10 x 6
1 x 5
3 x 5
6 x 3
2 x 4
6 x 3
7 x 2
8 x 3
4 x 2
4 x 2
10 x 6
7 x 2
8 x 3
Number of bins used: (1, [(-5, 10)])


In [21]:
def visualize_packing(bin_width, bin_height, items, bins):
    fig, ax = plt.subplots()

    # Plot the bins
    ax.set_xlim(0, bin_width)
    ax.set_ylim(0, bin_height)
    ax.set_aspect('equal')
    ax.set_title('Bin Packing')
    for bin in bins:
        ax.add_patch(plt.Rectangle((0, 0), bin_width, bin[0], fill=False, edgecolor='blue'))

    # Plot the packed items
    for bin in bins:
        x = []
        y = []
        for item in items:
            if item in bin[1]:
                x.append(item[0])
                y.append(item[1])
        ax.scatter(x, y, color='red')

    plt.show()

In [22]:
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')
num_bins = fbs_heuristic(bin_width, bin_height, items)
bins = [(bin_height, items[i:i+num_bins]) for i in range(0, len(items), num_bins)]
visualize_packing(bin_width, bin_height, items, bins)

TypeError: 'tuple' object cannot be interpreted as an integer

the visualize_packing function creates a visualization of the packing solution. It plots the bins as blue rectangles and the packed items as red dots. The read_dataset and fbs_heuristic functions are used to obtain the bin dimensions, items, and the number of bins used.

### Recherche locale 

In [23]:
def local_search(bin_width, bin_height, items, max_iterations=1000):
    # Initialize the solution with a random placement of items
    solution = [(random.randint(0, bin_width - 1), random.randint(0, bin_height - 1)) for _ in range(len(items))]

    # Evaluate the initial solution
    fitness = evaluate_solution(bin_width, bin_height, items, solution)

    # Perform local search
    for _ in range(max_iterations):
        # Generate a neighbor solution by moving one item to a new location
        neighbor_solution = generate_neighbor(solution, bin_width, bin_height)

        # Evaluate the neighbor solution
        neighbor_fitness = evaluate_solution(bin_width, bin_height, items, neighbor_solution)

        # If the neighbor solution is better, accept it
        if neighbor_fitness < fitness:
            solution = neighbor_solution
            fitness = neighbor_fitness

    return solution

def generate_neighbor(solution, bin_width, bin_height):
    # Select a random item to move
    item_index = random.randint(0, len(solution) - 1)

    # Generate a new location for the item
    new_x = random.randint(0, bin_width - 1)
    new_y = random.randint(0, bin_height - 1)

    # Create a new solution with the item moved to the new location
    neighbor_solution = solution[:item_index] + [(new_x, new_y)] + solution[item_index + 1:]

    return neighbor_solution

def evaluate_solution(bin_width, bin_height, items, solution):
    # Calculate the number of bins used
    bins_used = 0
    for item_index, (item_width, item_height) in enumerate(items):
        x, y = solution[item_index]
        if x + item_width > bin_width or y + item_height > bin_height:
            bins_used += 1

    # Calculate the waste in each bin
    waste = 0
    for item_index, (item_width, item_height) in enumerate(items):
        x, y = solution[item_index]
        waste += (bin_width - x - item_width) * (bin_height - y - item_height)

    # Return the fitness value (number of bins used + waste)
    return bins_used + waste / (bin_width * bin_height)

# def visualize_solution(bin_width, bin_height, items, solution):
#     fig, ax = plt.subplots()
#     ax.set_xlim(0, bin_width)
#     ax.set_ylim(0, bin_height)

#     for item_index, (item_width, item_height) in enumerate(items):
#         x, y = solution[item_index]
#         ax.add_patch(plt.Rectangle((x, y), item_width, item_height, fill=False))

#     plt.show()


In [24]:
# Load the dataset
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')

# Run the local search algorithm
solution = local_search(bin_width, bin_height, items)

# Print the solution
for item_index, (item_width, item_height) in enumerate(items):
    x, y = solution[item_index]
    print(f'Item {item_index}: ({x}, {y})')

Item 0: (5, 1)
Item 1: (0, 8)
Item 2: (0, 2)
Item 3: (4, 1)
Item 4: (1, 7)
Item 5: (0, 0)
Item 6: (9, 0)
Item 7: (7, 1)
Item 8: (4, 0)
Item 9: (8, 5)
Item 10: (4, 5)
Item 11: (1, 8)
Item 12: (2, 1)
Item 13: (0, 8)
Item 14: (6, 6)
Item 15: (0, 0)
Item 16: (3, 3)
Item 17: (2, 0)


In [25]:
# # Run the local search algorithm
# solution = local_search(bin_width, bin_height, items)

# # Visualize the solution
# visualize_solution(bin_width, bin_height, items, solution)

### Shaking/Diversification Procedure

In [26]:
def shake_solution(solution, max_shakes=10):
    for _ in range(max_shakes):
        # Select a random item to move
        item_index = random.randint(0, len(solution) - 1)

        # Generate a new location for the item
        new_x = random.randint(0, bin_width - 1)
        new_y = random.randint(0, bin_height - 1)

        # Create a new solution with the item moved to the new location
        solution[item_index] = (new_x, new_y)

    return solution

def diversify_solution(solution, max_diversifications=10):
    for _ in range(max_diversifications):
        # Select a random item to swap with another item
        item_index1 = random.randint(0, len(solution) - 1)
        item_index2 = random.randint(0, len(solution) - 1)

        # Swap the two items
        solution[item_index1], solution[item_index2] = solution[item_index2], solution[item_index1]

    return solution

In [27]:
# Load the dataset
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')

# Run the local search algorithm
solution = local_search(bin_width, bin_height, items)

# Shake the solution
print(shake_solution(solution))

# Diversify the solution
print(diversify_solution(solution))

[(2, 1), (5, 8), (1, 2), (5, 1), (4, 1), (3, 3), (8, 1), (5, 2), (1, 8), (8, 5), (4, 6), (3, 8), (2, 0), (3, 8), (3, 2), (0, 4), (6, 3), (2, 1)]
[(2, 1), (8, 5), (0, 4), (4, 1), (3, 3), (5, 1), (2, 0), (5, 2), (1, 8), (4, 6), (5, 8), (3, 2), (8, 1), (3, 8), (3, 8), (1, 2), (6, 3), (2, 1)]


### Basic Variable Neighborhood Search

In [28]:
def vns_search(bin_width, bin_height, items, max_iterations=1000):
    solution = local_search(bin_width, bin_height, items)
    best_fitness = evaluate_solution(bin_width, bin_height, items, solution)
    best_solution = solution

    for _ in range(max_iterations):
        # Shake the solution
        solution = shake_solution(solution)

        # Evaluate the solution
        fitness = evaluate_solution(bin_width, bin_height, items, solution)

        # Check if the solution is better than the current best
        if fitness < best_fitness:
            best_fitness = fitness
            best_solution = solution

        # Diversify the solution
        solution = diversify_solution(solution)

    return best_solution

# Load the dataset
bin_width, bin_height, items = read_dataset('instance2dBIN.txt')

# Run the VNS algorithm
best_solution = vns_search(bin_width, bin_height, items)

print(best_solution)

[(8, 4), (7, 8), (5, 4), (4, 8), (0, 3), (0, 5), (5, 7), (0, 8), (1, 3), (6, 0), (5, 9), (3, 0), (4, 8), (6, 3), (3, 6), (0, 9), (2, 4), (3, 9)]


In [29]:
# Evaluate the solution
def evaluate_solution(bin_width, bin_height, items, solution):
    bins_used = 0
    for item_index, (item_width, item_height) in enumerate(items):
        x, y = solution[item_index]
        if x + item_width > bin_width or y + item_height > bin_height:
            bins_used += 1
    waste = 0
    for item_index, (item_width, item_height) in enumerate(items):
        x, y = solution[item_index]
        waste += (bin_width - x - item_width) * (bin_height - y - item_height)
    return bins_used + waste / (bin_width * bin_height)

### Evaluation et Inference

In [30]:
# Main function to run and evaluate all algorithms
def main():
    bin_width, bin_height, items = read_dataset('instance2dBIN.txt')
    # FBS Heuristic
    start_time = time.time()
    num_bins, bins = fbs_heuristic(bin_width, bin_height, items)
    end_time = time.time()
    execution_time_fbs = end_time - start_time

    # Local Search
    start_time = time.time()
    solution_ls = local_search(bin_width, bin_height, items)
    fitness_ls = evaluate_solution(bin_width, bin_height, items, solution_ls)
    end_time = time.time()
    execution_time_ls = end_time - start_time

    # VNS
    start_time = time.time()
    solution_vns = vns_search(bin_width, bin_height, items)
    fitness_vns = evaluate_solution(bin_width, bin_height, items, solution_vns)
    end_time = time.time()
    execution_time_vns = end_time - start_time

    # Printing results
    print(f"FBS Heuristic: Number of bins used = {num_bins}, Execution time = {execution_time_fbs:.4f}s")
    print(f"Local Search: Fitness = {fitness_ls}, Execution time = {execution_time_ls:.4f}s")
    print(f"VNS: Fitness = {fitness_vns}, Execution time = {execution_time_vns:.4f}s")

if __name__ == "__main__":
    main()

FBS Heuristic: Number of bins used = 1, Execution time = 0.0000s
Local Search: Fitness = 0.0, Execution time = 0.0088s
VNS: Fitness = 10.83, Execution time = 0.0590s


# Analyse des Algorithmes de Bin Packing

## Introduction

Le problème de bin packing consiste à placer un ensemble d'objets de différentes tailles dans un nombre minimum de conteneurs (bins) de taille fixe. Les algorithmes heuristiques et de recherche sont couramment utilisés pour résoudre ce problème. Dans cette analyse, nous évaluons trois algorithmes : l'Heuristique FBS, la Recherche Locale, et la Recherche à Voisinage Variable (VNS).

## Algorithmes

### Heuristique FBS (Finite Best Strip)

L'heuristique FBS est un algorithme qui tente de minimiser le nombre de conteneurs en deux phases :
1. **Phase 1 : Emballage des objets en bandes de hauteur infinie**
    - Les objets sont triés par hauteur décroissante.
    - Chaque objet est placé sur l'étagère (shelf) où il reste le moins de largeur, sinon une nouvelle étagère est créée.
2. **Phase 2 : Emballage des étagères dans des conteneurs de taille finie**
    - Les étagères sont placées dans des conteneurs en essayant de minimiser l'espace inutilisé.

### Recherche Locale

La recherche locale est une méthode d'optimisation itérative qui cherche à améliorer une solution initiale par des modifications locales :
- **Initialisation** : Une solution initiale est générée en plaçant les objets aléatoirement dans les conteneurs.
- **Évaluation** : La solution est évaluée en termes de fitness, qui combine le nombre de conteneurs utilisés et l'espace gaspillé.
- **Recherche** : À chaque itération, une solution voisine est générée et évaluée. Si elle est meilleure, elle remplace la solution courante.

### Recherche à Voisinage Variable (VNS)

La recherche à voisinage variable combine des techniques de diversification et d'intensification :
- **Initialisation** : Utilise une solution de la recherche locale.
- **Shaking** : Perturbe la solution en déplaçant aléatoirement des objets pour explorer de nouvelles configurations.
- **Recherche Locale** : Applique la recherche locale à la solution perturbée.
- **Diversification** : Échange aléatoirement des objets entre différentes positions pour éviter les minima locaux.

## Données Utilisées

Les données suivantes ont été utilisées pour les tests :

```plaintext
W: 10
H: 10

w: 5, h: 9
w: 4, h: 2
w: 10, h: 6
w: 5, h: 7
w: 6, h: 3
w: 10, h: 7
w: 1, h: 5
w: 3, h: 5
w: 6, h: 9
w: 2, h: 4
w: 6, h: 7
w: 7, h: 2
w: 8, h: 3
w: 4, h: 10
w: 4, h: 5
w: 10, h: 3
w: 7, h: 8
w: 8, h: 7
```

## Résultats des Algorithmes

Les résultats obtenus sont les suivants :

- **Heuristique FBS** :
  - Nombre de conteneurs utilisés = 1
  - Temps d'exécution = 0.0000s

- **Recherche Locale** :
  - Fitness = 0.0
  - Temps d'exécution = 0.0088s

- **VNS** :
  - Fitness = 10.83
  - Temps d'exécution = 0.0590s

## Analyse des Résultats

### Heuristique FBS

L'heuristique FBS a été très efficace, utilisant un seul conteneur avec un temps d'exécution extrêmement rapide (0.0000s). Cela montre que l'algorithme est capable de trouver une solution optimale de manière très efficace pour les données fournies. Son approche en deux phases permet de minimiser le nombre de conteneurs tout en restant rapide.

### Recherche Locale

La recherche locale a également donné d'excellents résultats avec une valeur de fitness de 0.0, indiquant une optimisation complète de l'espace. Cependant, elle a pris plus de temps (0.0088s) par rapport à l'heuristique FBS. Cela est dû au fait que la recherche locale itère plusieurs fois pour améliorer la solution, ce qui prend plus de temps mais permet d'atteindre une solution très optimisée.

### VNS (Variable Neighborhood Search)

La VNS a montré une valeur de fitness de 10.83, suggérant qu'il y avait encore de l'espace gaspillé. Son temps d'exécution de 0.0590s est le plus long parmi les trois algorithmes testés. La VNS est une méthode puissante dans des contextes plus variés et complexes car elle combine diversification et intensification. Cependant, pour ce cas particulier de bin packing, elle s'est révélée moins efficace en termes de temps d'exécution et d'utilisation de l'espace.

## Conclusion

En résumé, l'Heuristique FBS s'avère être la plus performante en termes d'efficacité et de rapidité pour ce problème spécifique de bin packing. La recherche locale, bien qu'un peu plus lente, offre une excellente optimisation de l'espace. La VNS, bien que polyvalente et potentiellement plus robuste dans d'autres contextes, est moins optimale pour ce cas particulier en raison de son temps d'exécution plus long et de l'optimisation de l'espace moins efficace.

### Recommandations

- **Heuristique FBS** : À utiliser lorsque la rapidité est cruciale et que les données sont similaires à celles fournies.
- **Recherche Locale** : À utiliser lorsque l'optimisation de l'espace est une priorité, même au coût d'un temps de calcul légèrement plus élevé.
- **VNS** : À considérer pour des problèmes plus complexes où la robustesse et la flexibilité sont nécessaires, malgré un compromis sur le temps d'exécution et l'efficacité.